In [49]:
class Vendor_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in vendor event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
        
class GroundTruth_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in ground truth event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]




#Scoring by testing various permutations of individual fields
#thought process is the following
#Actor 1 will pull GT7,GT6,GT10,GT16 then 'empty'
#Actor 2 will pull GT7,GT6,GT10,GT16 then 'empty'
#Actor1mention  will pull GT7,GT6,GT10,GT16 then 'empty'
#Actor2mention  will pull GT7,GT6,GT10,GT16 then 'empty'
#Geo we will switch up to try GT10 GT16, GT7, GT6
#Cameo will be GT7,GT16, GT10, GT6
#clustering across cameo codes we need to be careful as to not cross cameos from different types or do we???
#
# 1.) read in all clusters from tamr
# 2.) find how many clusters we have
# 2a.) get distinct clusters
# 3.) start processing each cluster
#   3a.) read in custer[x]
#   3b.) set fields based on rules from above (how we do this)
#   3c.) print out record and where each vaue came from (how do we keep track of where everything came from)
#
#   3d.) score the record
#   3e.) get next cluster

# for i,facts in enumerator(clusters):
#  if facts[i][8] == 'gt7' && facts[i][8] not empty:
#    actor1 = facts[i][2]  <--- get actor1 from gt7 record
# otherwise we would have to check how many gt7 records exist in cluster and check each of those before moving on
import sys
import os
from munkres import Munkres
import numpy as np
import re
import csv
import itertools

import collections

# function to build our record to score
def buildrecord(tamr_records):

        
    #check how many instances of each vendor type we have
    gt7_count = sum(x.count('GT7') for x in tamr_records)
    gt16_count = sum(x.count('aic.GT16') for x in tamr_records)
    gt10_count = sum(x.count('GT10') for x in tamr_records)
    gt6_count = sum(x.count('GT6') for x in tamr_records)

    print gt7_count
    
    #for i,facts in enumerate(tamr_records):
        
        #check for GT7 existence and actor value present
    if gt7_count > 0:
        
        actor1= [e[1] for e in tamr_records if e[6]=='GT7']
        actor2= [e[2] for e in tamr_records if e[6]=='GT7']
        actor1mention= [e[3] for e in tamr_records if e[6]=='GT7']
        actor2mention= [e[4] for e in tamr_records if e[6]=='GT7']
        cameocode= [e[7] for e in tamr_records if e[6]=='GT7']

        if actor1:
            actor1_from = 'A1 from GT7'
        if actor2:
            actor2_from = 'A2 from GT7'
        if actor1mention:
            actor1mention_from = 'A1M from GT7'
        if actor2mention:
            actor2mention_from = 'A2M from GT7'
        if cameocode:
            cameocode_from = 'Cameo from GT7'
#check gt6
    if gt6_count > 0:
        
        if not actor1:
            actor1= [e[1] for e in tamr_records if e[6]=='GT6']
            if actor1:
                actor1_from = 'A1 from GT6'
        if not actor2:
            actor2= [e[2] for e in tamr_records if e[6]=='GT6']
            if actor2:
                actor2_from = 'A2 from GT6'
        if not actor1mention:
            actor1mention= [e[3] for e in tamr_records if e[6]=='GT6']
            if actor1mention:
                actor1mention_from = 'A1M from GT6'        
        if not actor2mention:
            actor2mention= [e[4] for e in tamr_records if e[6]=='GT6']
            if actor2mention:
                actor2mention_from = 'A2M from GT6'
        if not cameocode:    
            cameocode= [e[7] for e in tamr_records if e[6]=='GT6']
            if cameocode:
                cameocode_from = 'Cameo from GT6'
#check gt10
    if gt10_count > 0:
        
        if not actor1:
            actor1= [e[1] for e in tamr_records if e[6]=='GT10']
            if actor1:
                actor1_from = 'A1 from GT10'
        if not actor2:
            actor2= [e[2] for e in tamr_records if e[6]=='GT10']
            if actor2:
                actor2_from = 'A2 from GT10'
        if not actor1mention:
            actor1mention= [e[3] for e in tamr_records if e[6]=='GT10']
            if actor1mention:
                actor1mention_from = 'A1M from GT10'        
        if not actor2mention:
            actor2mention= [e[4] for e in tamr_records if e[6]=='GT10']
            if actor2mention:
                actor2mention_from = 'A2M from GT10'
        if not cameocode:    
            cameocode= [e[7] for e in tamr_records if e[6]=='GT10']
            if cameocode:
                cameocode_from = 'Cameo from GT10'
   
        geofullname = cameocode= [e[5] for e in tamr_records if e[6]=='GT10']
    
        if geofullname:
            geofullname_from = 'Geo from GT10'        
       

    if not actor1:
        actor1 = 'empty'
        actor1_from = 'from empty'

    if not actor2:
        actor2 = 'empty'
        actor2_from = 'from empty'
    
    if not actor1mention:
        actor1mention = 'empty'
        actor1mention_from = 'from empty'

    if not actor2mention:
        actor2mention = 'empty'
        actor2mention_from = 'from empty'
        
    if not cameocode:
        cameocode = 'empty'
        cameocode_from = 'from empty'
        
    if not geofullname:
        geofullname = 'empty'
        geofullname_from = 'from empty'
    
    record_to_score = []
    record_to_score.extend([actor1,actor2,actor2mention,cameocode,geofullname,actor1_from,actor2_from,actor1mention_from,actor2mention_from,cameocode_from,geofullname_from])
    print record_to_score
    #print actor1,actor2,actor1mention,actor2mention,cameocode,geofullname,actor1_from,actor2_from,actor1mention_from,actor2mention_from,cameocode_from,geofullname_from
        
    return record_to_score







#****************
#main body of program
#****************

#read in tamr data
#tamr_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfcluster1.score.csv'
tamr_data = '/Users/Pete/Customers/2016.Feb Greenfield/July21ClusterScoring/tamrcluster2.csv'


with open(tamr_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    tamrds = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

#read in ground truth data
gt_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfgroundtruthtest1.score.csv'
#gt_data = '/Users/Pete/Customers/gt.matchtest1'

with open(gt_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    dsgold = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

#print len(tamrds),tamrds
#print len(dsgold),dsgold

clusters_to_process = []

#get all the clusters
for counter,clusters in enumerate(tamrds):
    clusters_to_process.append(tamrds[counter][8])
    
clusters = list(set(clusters_to_process))    
print 'our unique cluster ids',clusters

#let's loop through and filter our data we don't need right now
for count,elements in enumerate(clusters):
       
    dstamr_clusters = filter(lambda tamrds: tamrds[8] in elements,tamrds)
    print 'records to process',dstamr_clusters
    
# for i,facts in enumerator(clusters):
#  if facts[i][8] == 'gt7' && facts[i][8] not empty:
#    actor1 = facts[i][2]  <--- get actor1 from gt7 record
# otherwise we would have to check how many gt7 records exist in cluster and check each of those before moving on

    #call function to build record to score from our cluster
    tamr_record_to_score = buildrecord(dstamr_clusters)
    
    print 'I am back', tamr_record_to_score[6]
    

our unique cluster ids ['78ccdf54-b6c4-41c3-b03e-8985a0c7c8be']
records to process [['400356', 'A European delegation on Friday', '', 'A European delegation on Friday', '', '', 'aic.gt16', '10', '78ccdf54-b6c4-41c3-b03e-8985a0c7c8be', 'aaa', '424178'], ['401812', 'AFP', 'interim President', '', '', 'Central African Republic', 'GT10', '10', '78ccdf54-b6c4-41c3-b03e-8985a0c7c8be', 'aaa', '153414'], ['402958', 'A European delegation on Friday', 'the Central African Republic', 'A European delegation on Friday', 'the Central African Republic', '', 'GT7', '331', '78ccdf54-b6c4-41c3-b03e-8985a0c7c8be', 'aaa', '5969584']]
1
[['A European delegation on Friday'], ['the Central African Republic'], ['the Central African Republic'], ['Central African Republic'], ['Central African Republic'], 'A1 from GT7', 'A2 from GT7', 'A1M from GT7', 'A2M from GT7', 'Cameo from GT7', 'Geo from GT10']
I am back A2 from GT7
